In [3]:
'''
    This below python code scrap website "https://www.glastonburyfestivals.co.uk/line-up/line-up-2019/" using beautifulsoup 
    library and create an excel sheet(by using xlsxwriter package) with all the records.
''' 

import requests
from bs4 import BeautifulSoup
import xlsxwriter

# Assinging url in a variable
url = "https://www.glastonburyfestivals.co.uk/line-up/line-up-2019/"

# Headers require to make get request for a third part url
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}
page = requests.get(url, headers)

# Assigning complete page content in a varibale called soup
soup = BeautifulSoup(page.content, 'html.parser')

# Excel workbook initialization
workbook = xlsxwriter.Workbook('Festival_Events_Csv.xlsx')
worksheet = workbook.add_worksheet()

# Widen the first column to make the text clearer.
worksheet.set_column('A:A', 20)

# Defining how mnay columns needed with names
worksheet.write('A1', 'name')
worksheet.write('B1', 'day')
worksheet.write('C1', 'start')
worksheet.write('D1', 'end')

# Finding all the tables  with class name "lineup_table_day"
all_tables = soup.find_all('table', class_='lineup_table_day')

i = 0

# Initialise excel_row_count varibale with 2 becasue first rows is a column names of excel sheet
excel_row_count = 2
while i < len(all_tables):
    k = 0
    
    # Day Name of Each Table
    day_name = all_tables[i].find("h2")
    
    # Storing Day Name in a sheet in column A
    worksheet.write('A' + str(excel_row_count), day_name.get_text())
    
    # Finding all the tr inside single table 
    first_table_content = all_tables[i].find_all("tr")
    
    # Removing first tr as it doesnt contain any table, instead it has h2 tag (Day Name)
    first_table_content = first_table_content[1:]
    
    # Loop to get all the event names with timings
    while k < len(first_table_content):
        timings = first_table_content[k].find_all('td')
        timings = timings[1].get_text()
        
        # spliting the time in start and end Time
        timings = timings.split(" - ")
        
        # Add day name title in Sheet
        worksheet.write('A' + str(excel_row_count), day_name.get_text().title())
        
        # Handling exception where event name is not link
        try:
            worksheet.write('B' + str(excel_row_count), first_table_content[k].find("a").get_text().title())
        except:
            name = first_table_content[k].find("td").get_text()
            worksheet.write('B' + str(excel_row_count), name.title())
        
        # Handling exception where timing is not available
        try:
            worksheet.write('C' + str(excel_row_count), timings[0])
        except:
            worksheet.write('C' + str(excel_row_count), "")
        try:
            worksheet.write('D' + str(excel_row_count), timings[1])
        except:
            worksheet.write('D' + str(excel_row_count), "")
        excel_row_count = excel_row_count+1
        k = k+1
    i = i+1
workbook.close()